In [8]:
from matdeeplearn.common import data
import os
import torch
import matplotlib.pyplot as plt
from pprint import pprint

from matdeeplearn.preprocessor.helpers import *
from matdeeplearn.preprocessor.transforms import *
from matdeeplearn.common.graph_data import CustomData

from torch_geometric.loader import DataLoader


from ase import neighborlist

%pprint
%load_ext autoreload
%autoreload 2

torch.set_printoptions(profile="full")

Pretty printing has been turned ON
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
dataset = data.get_dataset("/nethome/sbaskaran31/projects/Sidharth/hMOF/raw_5k/ocp")

In [10]:
sample = dataset[1]

In [13]:
sample.cell_offset_distances

AttributeError: 'GlobalStorage' object has no attribute 'cell_offset_distances'

In [4]:
# params
n_neighbors = 50
r = 5.0

In [5]:
 # Compute graph using ASE method
(
    first_idex,
    second_idex,
    rij,
    rij_vec,
    shifts,
) = ase.neighborlist.primitive_neighbor_list(
    "ijdDS",
    (True, True, True),
    ase.geometry.complete_cell(sample.cell),
    sample.pos.numpy(),
    cutoff=r,
    self_interaction=True,
    use_scaled_positions=False,
)

In [6]:
from torch_geometric.utils import dense_to_sparse

In [7]:
edge_weights = torch.tensor(rij).float()
first_idx, second_idx = torch.tensor(first_idex), torch.tensor(second_idex)

In [22]:
edge_weights.count_nonzero()

tensor(9986)

In [8]:
edge_weights.shape

torch.Size([10122])

In [18]:
distance_matrix = torch.zeros((sample.n_atoms, sample.n_atoms))
# fill with infinite distance for non-edges
# distance_matrix.fill_(np.inf)
distance_matrix[first_idx, second_idx] = edge_weights

In [21]:
sample.n_atoms

tensor([136])

In [19]:
distance_matrix.count_nonzero()

tensor(9914)

Number of missing elements are zero distance edges, which are implicitly remove when converted to dense representation

In [11]:
_, indices = torch.topk(distance_matrix, k=50, largest=False, dim=1)

In [12]:
indices.shape

torch.Size([136, 50])

In [13]:
# marks which edges are within the neighbor threshold
mask = torch.zeros_like(distance_matrix).scatter(1, indices, 1)

In [14]:
distance_matrix_masked = distance_matrix * mask
finite_mask = torch.isfinite(distance_matrix_masked)
distance_matrix_masked = distance_matrix_masked.masked_fill(~finite_mask, 0)

In [15]:
edge_index_new, edge_weights_new = dense_to_sparse(distance_matrix_masked)

In [16]:
edge_index_new.shape

torch.Size([2, 6664])

Figure out how to get indices of the top neighbors from new edge_index

In [17]:
edge_index = torch.stack([first_idx, second_idx], dim=1)
edge_index_new = edge_index_new.transpose(0,1)